In [9]:
# NOTICE!!!!
# replace 'E' in the equation with 'e' as sympy would treat 'E' as numerical value e.
##################

import sympy as sp
import pandas as pd

# Define custom operations using subclassing of sympy.Function
class Add(sp.Function):
    @classmethod
    def eval(cls, x, y):
        return x + y

class Sub(sp.Function):
    @classmethod
    def eval(cls, x, y):
        return x - y

class Mul(sp.Function):
    @classmethod
    def eval(cls, x, y):
        return x * y

class Div(sp.Function):
    @classmethod
    def eval(cls, x, y):
        try:
            result = sp.Piecewise(
                (1, sp.Eq(y, 0)),  # Return 1 if y is exactly zero
                (1, sp.Eq(y, sp.zoo)),  # Return 1 if y is exactly zero
                (x / y, True)           # Normal division otherwise
            )
            return result.simplify()  # Simplify to handle potential further issues
        except Exception as e:
            print(f"Division error: {e}")
            return 1000  # Return NaN on error
        
# Define the custom operations using subclassing of sympy.Function
class Div_for_visualization(sp.Function):
    @classmethod
    def eval(cls, x, y):
        # Use Piecewise to handle division by zero robustly
        return x / y

class Neg(sp.Function):
    @classmethod
    def eval(cls, x):
        return -x
    

# Update converter for sympify to recognize custom classes
converter = {
    'add': Add,
    'sub': Sub,
    'mul': Mul,
    'div': Div,
    'neg': Neg,
}

# Update converter for sympify to recognize custom classes
converter_vis = {
    'add': Add,
    'sub': Sub,
    'mul': Mul,
    'div': Div_for_visualization,
    'neg': Neg,
}

# Function to apply the equation to each row of the DataFrame
def apply_equation(row):
    # Replace variables in the expression with row data
    local_env = {
        'Pr': row['Pr'],
        'La': row['La'],
        'Gd': row['Gd'],
        'Nd': row['Nd'],
        'Ce': row['Ce'],
        'Sm': row['Sm'],
        'Y' : row['Y'],
        "EN": row['EN'],
        'R' : row['R'],
        'IE': row['IE'],
        'LAS': row['LAS'],
        'M' : row['M'],
        'e' : row['E'],
    }
    try:
        result = expr.subs(local_env)
        # Force numeric evaluation if the result is not numeric
        if isinstance(result, sp.Expr):  # Check if result is still a symbolic expression
            result = result.evalf()  # Evaluate numerically
        # if result == sp.zoo:
        #     result = 1
    except Exception as e:
        print(f"Error evaluating row: {local_env}")
        print(f"Error message: {str(e)}")
        result = None  # Optional: handle error case by assigning a default value or None
    return result

    # print("Local Environment:", local_env)  # Debug: Print the local environment for a row
    # return expr.subs(local_env)

In [10]:
equ = "add(div(sub(Pr, LAS), 3.357), add(div(sub(Nd, 4.782), mul(sub(R, Y), mul(IE, 3.052))), e))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./all_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('all_rp700_pred.xlsx', index=False)

-0.297885016383676*LAS + 0.297885016383676*Pr + e + 0.327653997378768*(Nd - 4.782)/(IE*(R - Y))


In [11]:
equ = "add(div(e, add(div(sub(EN, IE), sub(IE, R)), mul(div(EN, Ce), Nd))), EN)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./La_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('La_rp700_pred.xlsx', index=False)

EN + e/((EN - IE)/(IE - R) + EN*Nd/Ce)


In [12]:
equ = "div(mul(IE, add(div(div(sub(e, EN), sub(Sm, EN)), div(La, div(Nd, La))), e)), Y)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Gd_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Gd_rp700_pred.xlsx', index=False)

IE*(e + Nd*(-EN + e)/(La**2*(-EN + Sm)))/Y


In [13]:
equ = "add(div(sub(M, La), 1.633), add(div(sub(R, Sm), sub(2.403, mul(2.922, sub(Gd, Y)))), e))"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Nd_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Nd_rp700_pred.xlsx', index=False)

-0.612369871402327*La + 0.612369871402327*M + e + (R - Sm)/(-2.922*Gd + 2.922*Y + 2.403)


In [14]:
equ = "add(sub(e, add(div(sub(e, La), div(mul(-6.635, sub(e, La)), sub(La, Gd))), LAS)), EN)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Ce_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Ce_rp700_pred.xlsx', index=False)

EN - LAS + e - (-Gd + La)*(-La + e)/(6.635*La - 6.635*e)


In [15]:
equ = "add(div(sub(M, Y), sub(La, sub(add(div(sub(Nd, Y), sub(M, Gd)), 6.465), La))), e)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter,evaluate=True)
# Read the Excel file
df = pd.read_excel('./Sm_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Sm_rp700_pred.xlsx', index=False)

e + (M - Y)/(2*La - 6.465 - (Nd - Y)/(-Gd + M))


In [16]:
equ = "add(div(sub(sub(LAS, -5.767), add(e, EN)), mul(sub(IE, La), 3.836)), e)"
expr = sp.sympify(equ, locals=converter_vis)
print(expr)

expr = sp.sympify(equ, locals=converter)

# Read the Excel file
df = pd.read_excel('./Y_rp700.xlsx')
df = df.apply(lambda x: round(x, 4) if x.dtype.kind in 'fc' else x)

# Apply the function to each row
df['Pred'] = df.apply(apply_equation, axis=1)
# Save the result back to a new Excel file
df.to_excel('Y_rp700_pred.xlsx', index=False)
# df['Pred']

e + (-EN + LAS - e + 5.767)/(3.836*IE - 3.836*La)
